In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [2]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

# NASA Mars News

In [3]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

In [4]:
newsTitleList = soup.find_all('div', {'class': 'content_title'})
newsTextList = soup.find_all('div', {'class': 'article_teaser_body'})

# Indexed at one because the first item in the list isn't the correct title.
newsTitle = newsTitleList[1].text
newsText = newsTextList[0].text

print(newsTitle)
print('\n')
print(newsText)


NASA Moves Forward With Campaign to Return Mars Samples to Earth


During this next phase, the program will mature critical technologies and make critical design decisions as well as assess industry partnerships.


# JPL Mars Space Images - Featured Image

In [5]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

In [6]:
relative_image_path = soup.find('a', {'class':'fancybox'})["data-fancybox-href"]
images_url = 'https://www.jpl.nasa.gov'
featured_img = images_url + relative_image_path
print(featured_img)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14293_ip.jpg


# Mars Facts

In [21]:
url = "https://space-facts.com/mars/"
browser.visit(url)

time.sleep(1)

table = pd.read_html(url)
print(table)

[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers,   Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2                1
3      Distance from Sun:   227,943,824 km   149,598,262 km
4         Length of Year:   687 Earth days      365.24 days
5            Temperature:     -87 to -5 °C      -88 to 58°C,                       0    

In [25]:
marsFacts_df = table[0]
marsFacts_df.columns = ['Description', 'Value']
htmlString = marsFacts_df.to_html()
# print(htmlString)

# Mars Hemispheres

In [37]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

In [44]:
imageDivs = soup.find_all('div', {'class': 'description'})

images = []

baseURL = 'https://astrogeology.usgs.gov'

for div in imageDivs:
    title = div.find('h3').text
    
    imagePath = div.find('a')['href']
    imageURL = baseURL + imagePath
    
    browser.visit(imageURL)
    imageHTML = browser.html
    imageSoup = bs(imageHTML, "html.parser")
    
    imageLink = baseURL + imageSoup.find('img', {'class': 'wide-image'})['src']
    
    images.append({'title': title, 'img_url': imageLink})
    
print(images)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
